## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.layers import Dense, Conv2D, MaxPool2D, Activation, Dropout, Flatten
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.datasets import cifar10
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 使用 CPU

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 539430479022924295, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3188470579
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18139284467487079018
 physical_device_desc: "device: 0, name: GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0"]

In [3]:
num_class = 10
BATCH_SIZE = 128
EPOCHS = 15

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

y_train = keras.utils.to_categorical(y_train, num_class)
y_test = keras.utils.to_categorical(y_test, num_class)

In [5]:
print(x_train.shape)

(50000, 32, 32, 3)


In [6]:
# ======== EarlyStop =======
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [7]:
# ======== ModelCheckPoint ========
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='./resnet_cifar10.h5', monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
# =========== ResNet50 ============
model = ResNet50(input_shape=(32,32,3), weights='imagenet', pooling='max', include_top=False)

res_featuremap = model.output
# flatten_featuremap = Flatten()(res_featuremap)
output = Dense(units=num_class, activation='softmax')(res_featuremap)

model = keras.models.Model(input=[model.input], output=[output])

In [8]:
model = keras.models.load_model('./resnet_cifar10.h5')
loss_loadback, acc_loadback = model.evaluate(x_test, y_test)
print('loss: ', loss_loadback)
print('accuracy: ', acc_loadback)

10000/10000 [==============================] - 18s 2ms/step
loss:  0.7219919486045837
accuracy:  0.7684999704360962


In [9]:
model.compile(loss='categorical_crossentropy', 
                  optimizer=keras.optimizers.Adam(), 
                  metrics=['accuracy'])

In [15]:
from keras.preprocessing.image import ImageDataGenerator

# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

In [16]:
model.fit_generator(data_generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=int(len(x_train)/BATCH_SIZE),
                    epochs=EPOCHS, 
                    validation_data=(x_test, y_test), 
                    verbose=1,
                    callbacks=[checkpoint],
                    shuffle=True)

scores = model.evaluate(x_test, y_test, verbose=0)
print('loss: ', scores[0])
print('accuracy: ', scores[1])

Epoch 1/15
390/390 [==============================] - 174s 447ms/step - loss: 0.5795 - accuracy: 0.7984 - val_loss: 0.6849 - val_accuracy: 0.7654

Epoch 00001: val_loss did not improve from 0.65802
Epoch 2/15
390/390 [==============================] - 181s 465ms/step - loss: 0.5525 - accuracy: 0.8083 - val_loss: 0.7878 - val_accuracy: 0.7457

Epoch 00002: val_loss did not improve from 0.65802
Epoch 3/15
390/390 [==============================] - 184s 471ms/step - loss: 0.5677 - accuracy: 0.8037 - val_loss: 0.5937 - val_accuracy: 0.7999

Epoch 00003: val_loss improved from 0.65802 to 0.59372, saving model to ./resnet_cifar10.h5
Epoch 4/15
390/390 [==============================] - 184s 473ms/step - loss: 0.5750 - accuracy: 0.8022 - val_loss: 1.1759 - val_accuracy: 0.7173

Epoch 00004: val_loss did not improve from 0.59372
Epoch 5/15
390/390 [==============================] - 185s 474ms/step - loss: 0.5357 - accuracy: 0.8157 - val_loss: 0.6540 - val_accuracy: 0.7843

Epoch 00005: val_los

In [17]:
model = keras.models.load_model('./resnet_cifar10.h5')
loss, acc = model.evaluate(x_test, y_test)
print('loss: ', loss)
print('accuracy: ', acc)

10000/10000 [==============================] - 16s 2ms/step
loss:  0.584746712398529
accuracy:  0.807200014591217
